# 100. Non-Personalized Recommendation - Best Seller 추천

## 사용자 데이터 (u.user) 파일을 DataFrame으로 읽기 

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


## 영화 데이터 (u.item) 파일을 DataFrame으로 읽기
- title : 제목  
- release data : 개봉 날짜  
- 장르 : unknown에서 Western까지 19가지 변수 (0, 1)

In [3]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## 사용자의 영화평점 데이터 (u.data) 파일을 DataFrame으로 읽기

- 사용자-영화-평점으로 구성

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.rating', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


# Best Seller  추천 방식

- 모든 사람에게 가장 인기 있는 동일한  item 추천   
- 각 item 에 대한 평가를 평균하여 높은 평균값 순으로 추천

In [5]:
#각 영화에 대한 평가를 평균
movie_mean_rating = ratings.groupby('movie_id').mean()['rating']
movie_mean_rating.head()

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64

In [6]:
# 각 영화의 평균 평가를 내림차순으로 sort
movie_sort = movie_mean_rating.sort_values(ascending=False)[:5]
movies_best = movies.loc[movie_sort.index]
movies_best.title

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## 추천 시스템의 정확도 측정

$$RMSE=\sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y_i})^2}$$

## Best Seller 추천 방식의 정확도 계산

In [7]:
# movie id  별 평균 rating
movie_mean_rating.head(3)

movie_id
1    3.878319
2    3.206107
3    3.033333
Name: rating, dtype: float64

In [8]:
# 특정 user가 특정 movie_id 에 부여한 rating
ratings.loc[196, ['movie_id', 'rating']].head(3)

,movie_id,rating
user_id,,
196,242,3
196,393,4
196,381,4


각 user가 실제로 평가한 값 (y_true)과 그 영화의 평균 rating (y_pred)의 MSE 측정

In [9]:
rmse = []
for user in set(ratings.index):
    y_true = ratings.loc[user, 'rating']  
    y_pred = movie_mean_rating.loc[ratings.loc[user, 'movie_id']]
    assert len(y_true) == len(y_pred), "length different"
    accuracy = mean_squared_error(y_true, y_pred, squared=True)
    rmse.append(accuracy)
    
print("Best Seller 추천 방식의 RMSE = ", np.mean(rmse))

Best Seller 추천 방식의 RMSE =  1.0435720790919452
